In [ ]:
!pip install --upgrade pip
!pip install --upgrade transformers


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
from transformers import pipeline
import torch

# Use "cuda:0" if you have a CUDA-enabled GPU, otherwise use "cpu"
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small", device="cuda:0" if torch.cuda.is_available() else "cpu")

######check the results of openai/whisper-small before the fine tuning by coparing the results with the dataset Rev3_9643_2021.xls ###

In [ ]:
import os
import pandas as pd

# Load the Excel file
input_excel_path = "/content/drive/MyDrive/Rev3_9643_2021.xlsx"
df = pd.read_excel(input_excel_path,header=None)

# Update the first column with the full file path
base_folder = "/content/drive/MyDrive/wav_data"  # Replace with the base folder containing the subfolders

def construct_path(file_name):
    if file_name.count('-') != 1:  # Skip rows where filename has more or less than one '-'
        return None  # Set to None to indicate skipping
    folder_name, _ = file_name.split('-')  # Split by '-' to get the folder name
    file_path = os.path.join(base_folder, folder_name, file_name)
    return file_path

df.iloc[:, 0] = df.iloc[:, 0].apply(construct_path)

# Save the updated DataFrame back to Excel
output_excel_path = "updated_Rev3_9643_2021.xlsx"  # Replace with the desired output path
df.to_excel(output_excel_path, index=False)

print(f"Updated file saved to {output_excel_path}")


In [ ]:
df.head()

In [ ]:
!pip install pydub
import os
import pandas as pd
from pydub import AudioSegment

# Load the Excel file
input_excel_path = "updated_Rev3_9643_2021.xlsx" # Replace with your .xlsx file path
df = pd.read_excel(input_excel_path, header=None)
df.head()

In [ ]:
!pip install pydub
import os
import pandas as pd
from pydub import AudioSegment

# Load the Excel file
input_excel_path = "updated_Rev3_9643_2021.xlsx" # Replace with your .xlsx file path
df = pd.read_excel(input_excel_path, header=None)

# Base folder containing the audio files
base_folder = "/content/drive/MyDrive/wav_data"  # Replace with your base folder

# Get the first column name
first_column = df.columns[0]
output_audio_folder = "converted_audio"  # Replace with your desired folder
os.makedirs(output_audio_folder, exist_ok=True)  # Create folder if it doesn't exist

# Function to load audio as AudioSegment
def load_audio(file_path,index):
    try:
        file_path = file_path.strip()  # Remove extra spaces or newline characters
        audio_data = AudioSegment.from_file(file_path)
        new_audio_path = os.path.join(output_audio_folder, f"audio_{index}.wav")
        audio_data.export(new_audio_path, format="wav")
        return new_audio_path
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")
        return None

# Replace the paths with audio data or mark for removal
df[first_column] = df[first_column].apply(
    lambda x: load_audio(x, df.index[df[first_column] == x][0]) if isinstance(x, str) and os.path.exists(x.strip()) else None
)

# Remove rows with missing audio data
df = df.dropna(subset=[first_column])

# Rename columns to "audio" and "sentence"
df.columns = ["audio", "sentence"]

# Save the updated dataset
output_excel_path = "updated_audio_dataset.xlsx"  # Replace with your desired output file path
df.to_excel(output_excel_path, index=False)

print(f"Updated dataset saved to {output_excel_path}")


In [ ]:
!pip install datasets
from datasets import Dataset
from datasets import Audio
import pandas as pd

# Path to the Excel file
input_excel_path = "updated_audio_dataset.xlsx"  # Replace with your Excel file path

# Read the Excel file
df = pd.read_excel(input_excel_path)

# Ensure the DataFrame has the correct columns
assert "audio" in df.columns and "sentence" in df.columns, "The Excel file must contain 'audio' and 'sentence' columns."

# Structure data for Hugging Face Dataset
data = {
    "audio": df["audio"].tolist(),  # List of audio file paths
    "sentence": df["sentence"].tolist()  # List of text sentences
}

# Create the Hugging Face Dataset
dataset = Dataset.from_dict(data)

# Cast the "audio" column to Audio type with the desired sampling rate
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

# Print the dataset structure
print(dataset)

# Optionally, save the dataset to Hugging Face's DatasetDict or push to the hub
# For now, save locally as Arrow format



In [ ]:
from huggingface_hub import login

login()


In [ ]:
dataset.push_to_hub("updated_Rev3_9643_2021")